# WeatherPy - Hotel Search

### Dependencies and data

In [1]:
%matplotlib inline

import datetime as dt
import gmaps
import requests
import numpy as np
import pandas as pd

import config

In [2]:
# Settings
gmaps.configure(api_key=config.GMAPS_API_KEY)

In [3]:
# Load data
weather = pd.read_csv('data/city_weather.csv')
weather.head(2)

,ID,City,Country,Time,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Description,Rain,Snow
0,0,Garoowe,SO,2020-11-05 13:21:54,8.41,48.48,86.65,22,0,2.53,clear sky,0.0,0.0
1,1,Vendas Novas,PT,2020-11-05 13:21:54,38.68,-8.46,63.00,59,20,14.99,few clouds,0.0,0.0


### Find hotels near destinations within the user's preferred weather

In [4]:
min_mt, max_mt = weather['Max Temp'].min(), weather['Max Temp'].max()
min_mt, max_mt

(-15.68, 107.6)

In [5]:
def take_input(prompt, min_temp=min_mt, max_temp=max_mt, yes_no=False):
    
    '''
    Take user input for weather preferences.
    
    Args:
        prompt (str) - input prompt
        min_temp (float) - minimum temperature in data
        max_temp (float) - maximum temperature in data
        yes_no (bool) - weather the input should be "yes" or "no" only
        
    Returns:
        [float or str]
    '''
    
    i = 'input'
    n = i.isnumeric()
    
    if yes_no:
        while i not in ['yes', 'no']:
            i = input(prompt)
        i = 1 if i == 'yes' else 0
    else:
        while not n:
            try:
                i = float(input(prompt))
            except:
                continue
            if min_temp <= i <= max_temp:
                n = True
                
    return i
        
    
# Get preferred temperature range
min_temp = take_input('What is the lowest temperature you would like at your destination? ')
max_temp = take_input('What is the highest temperature you would like at your destination? ', min_temp=min_temp)
rain = take_input('Would you like it to rain at your destination? (yes/no) ', yes_no=True)
snow = take_input('Would you like it to snow at your destination? (yes/no) ', yes_no=True)
min_temp, max_temp, rain, snow

What is the lowest temperature you would like at your destination?  70
What is the highest temperature you would like at your destination?  90
Would you like it to rain at your destination? (yes/no)  no
Would you like it to snow at your destination? (yes/no)  no


(70.0, 90.0, 0, 0)

In [6]:
# Filter for max temp preference
pref_cities = weather.loc[(weather['Max Temp'] >= min_temp) & (weather['Max Temp'] <= max_temp)]

# Filter for rain preference
if rain:
    pref_cities = pref_cities[pref_cities['Rain'] > 0]
else:
    pref_cities = pref_cities[pref_cities['Rain'] == 0]

# Filter for snow preference
if snow:
    pref_cities = pref_cities[pref_cities['Snow'] > 0]
else:
    pref_cities = pref_cities[pref_cities['Snow'] == 0]
    
pref_cities.shape

(312, 13)

In [7]:
# Initialize dataframe
hotels = pref_cities.copy()
hotels['Hotel'] = np.NaN

# Request url
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
payload = {
    'key': config.GMAPS_API_KEY,
    'type': 'lodging',
    'radius': 5000
}

hotels.head(2)

,ID,City,Country,Time,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Description,Rain,Snow,Hotel
0,0,Garoowe,SO,2020-11-05 13:21:54,8.41,48.48,86.65,22,0,2.53,clear sky,0.0,0.0,NaN
3,3,Makakilo City,US,2020-11-05 13:21:55,21.35,-158.09,77.00,94,90,3.36,overcast clouds,0.0,0.0,NaN


In [8]:
# Example response
payload['location'] = f'{hotels.iloc[0, 4]},{hotels.iloc[0, 5]}'
ex_response = requests.get(base_url, params=payload)
ex_response.json()

{'html_attributions': [],
 'next_page_token': 'ATtYBwLBD0aCDqQCpj4LfhGZLfkl2-LrKeMWP7fieqa-aYB6VDqmX52Ti9To4mS0Kt0g38yofLr73OGCCM4YUeWFGGGOVo_lOFVPCZ2R26fizUqwii5lWq4S8svbFA7hHPSB1qKGcCnLnuhkbFdhoEKKN9pfra2jHo6-cB1ql4-Lof1ZwWybtFUhKdVGMEADI52IFn0631U7h35qL6LYRGxbvGrh6b27R9FyjHLXNsr7IzbrE6krzLb20EXABSQZp5neT8YOi5s8f1vyILhvjfC-pw25qwu1DmBXoGMr1FuBi1KE1EUIfOrGyhsvuGPXWt3MIolacXs3ZaaVZH9ztmAUdso7M2z2VY7bmW_R-yO1jSBHA5owd5snfNQm5lCRPfEYNNLl6a24w37Lf2lwbIA8Qkz1DKRlGbMRAyM3goedE2aLLOkVoZkY1Ll2',
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 8.407393699999998, 'lng': 48.4978782},
    'viewport': {'northeast': {'lat': 8.4088532302915,
      'lng': 48.4992459802915},
     'southwest': {'lat': 8.406155269708496, 'lng': 48.4965480197085}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
   'name': 'Curubo Hotel',
   'photos': [{'height': 456,
     'html_attributions': ['<a href="https://maps.google.com/maps/contrib/102

In [9]:
# Cities without results
cities404 = []

# Find nearest hotel for each city
for idx, row in hotels.iterrows():
    payload['location'] = f'{row["Latitude"]},{row["Longitude"]}'
    try:
        response = requests.get(base_url, params=payload)
        response_json = response.json()
        hotels.loc[idx, 'Hotel'] = response_json['results'][0]['name']
    except:
        cities404.append(idx)

print(len(cities404))

21


In [10]:
# Drop cities without results
hotels.dropna(inplace=True)
print(hotels.shape)
hotels.head(2)

(290, 14)


,ID,City,Country,Time,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Description,Rain,Snow,Hotel
0,0,Garoowe,SO,2020-11-05 13:21:54,8.41,48.48,86.65,22,0,2.53,clear sky,0.0,0.0,Curubo Hotel
3,3,Makakilo City,US,2020-11-05 13:21:55,21.35,-158.09,77.00,94,90,3.36,overcast clouds,0.0,0.0,Marriott's Ko Olina Beach Club


### Temperature heatmap

In [11]:
!jupyter labextension install @jupyter-widgets/jupyterlab-manager
!jupyter lab build

Config option `kernel_spec_manager_class` not recognized by `InstallLabExtensionApp`.
Building jupyterlab assets (build:prod:minimize)
[LabBuildApp] WARNING | Config option `kernel_spec_manager_class` not recognized by `LabBuildApp`.
[LabBuildApp] JupyterLab 2.2.8
[LabBuildApp] Building in /Users/tribui/miniconda3/envs/minds/share/jupyter/lab
[LabBuildApp] Building jupyterlab assets (build:prod:minimize)


In [12]:
!jupyter labextension list
!conda list

Config option `kernel_spec_manager_class` not recognized by `ListLabExtensionsApp`.
JupyterLab v2.2.8
Known labextensions:
   app dir: /Users/tribui/miniconda3/envs/minds/share/jupyter/lab
        @jupyter-widgets/jupyterlab-manager v2.0.0  enabled  OK
        @lckr/jupyterlab_variableinspector v0.5.1  enabled  OK
        jupyter-gmaps v0.9.0  enabled  OK
        jupyter-matplotlib v0.7.4  enabled  OK
        jupyterlab-flake8 v0.6.1  enabled  OK
        jupyterlab-interactive-dashboard-editor v0.3.1  enabled  OK

Uninstalled core extensions:
    @acmore/jupyterlab-spark-ui-tab
    @j123npm/jupyterlab-dash
    jupyterlab-chart-editor
    jupyterlab-dash
    jupyterlab-plotly
    jupyterlab_spark
    jupyterlab_sparkmonitor
    jupyterlab_tensorboard
    plotlywidget
# packages in environment at /Users/tribui/miniconda3/envs/minds:
#
# Name                    Version                   Build  Channel
_py-xgboost-mutex         2.0                       cpu_0    conda-forge
alembic        

In [13]:
# Locations and measurements
locations = hotels[['Latitude', 'Longitude']]
weights = hotels['Max Temp'].apply(lambda t: max(t, 0))

# Info
hotel_info = '''
<dl>
<dt>Hotel</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather</dt><dd>{Max Temp} F - {Description}</dd>
</dl>
'''

# Layers
heat_layer = gmaps.heatmap_layer(locations, weights, dissipating=False, point_radius=4, max_intensity=100)
mark_layer = gmaps.marker_layer(locations,
                                info_box_content=[hotel_info.format(**row) for idx, row in hotels.iterrows()])

# Heatmap
fig = gmaps.figure(center=(0.0, 0.0), zoom_level=1.75)
fig.add_layer(heat_layer)
fig.add_layer(mark_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [45]:
# Save data
hotels.to_csv('data/hotels.csv', index=False)
pd.read_csv('data/hotels.csv').head(2)

,ID,City,Country,Time,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Description,Rain,Snow,Hotel
0,1,Vendas Novas,PT,2020-11-05 13:21:54,38.68,-8.46,63.0,59,20,14.99,few clouds,0.0,0.0,Hotel Acez
1,3,Makakilo City,US,2020-11-05 13:21:55,21.35,-158.09,77.0,94,90,3.36,overcast clouds,0.0,0.0,Marriott's Ko Olina Beach Club
